# A quantum algorithm


Bernstein-Vazirani is an example of a problem that a quantum algorithm can solve in significantly less number of steps than any classical solution.

Not a practical problem, though, but easy to understand.



Given:

$$ f_h(x) = h \cdot x \mod 2 $$

Where:

  * $h$: is the hidden variable
  * $x$: is the input variable
  * $\cdot$: is the *dot* product of $x$ and $h$, i.e. for register of size $n$:
    $$ \sum_i^n = h_i \times x_i $$
    
Find $h$.

Suppose someone gives you an oracle `f` for a hidden number `h`. The oracle receives an integer `x` and returns:

* **1**: if the number of bits in the input that match the bits of the hidden value is even.
* **0**: if the number of bits in the input that match the bits of the hidden value is odd.


The problem is to find the value of `h`.



As an example, this is the output of oracle the oracle for $n$=3, $h$=6 (`[110]`) and all possible $x$:

$$
\begin{array}{|c|c|c|}
x & x_b (bits) & f_h(x) \\ 
\hline 
0 & 000 & 0 \\
1 & 001 & 0 \\
2 & 010 & 1 \\
3 & 011 & 1 \\
4 & 100 & 1 \\
5 & 101 & 1 \\
6 & 110 & 0 \\
7 & 111 & 0 \\
\end{array}
$$





For registers of size $n$, **the best classical solution requires the code to call the oracle $n$ times** to find the hidden value $h$. It goes by this:

```
for i in 0 .. n-1
   Call the oracle with the number x that only has the i-th bit on
   if the oracle returns 1, the i-th bit of h is 1, otherwise it is 0

```

For example, if `h=6 [110]` and `n=3`:

```
h_0 = f(001) // returns 0
h_1 = f(010) // returns 1
h_2 = f(100) // returns 1

h = [110]     // as expected
```



In Q#, you can implement a function that creates this classical oracle with this code: 

In [1]:
// Required for IntAsBoolArray
open Microsoft.Quantum.Convert;

// Implements the classical oracle for the hidden input variable
function ClassicOracle(n: Int, hidden: Int, x: Int) : Bool {
    // Get the bits of the int as an array:
    let h_bits = IntAsBoolArray(hidden, n);
    let x_bits = IntAsBoolArray(x, n);

    mutable result = 0;

    // Implement the dot product of the bits:
    for i in 0 .. n-1 {
        if (h_bits[i] and x_bits[i]) {
            set result += 1;
        }
    }

    // Print a message on the console to report the oracle was called.
    Message($"Oracle C (h:{hidden}, x:{x}) - {result}.");

    // Return the mod 2 of the result:
    return (result % 2) == 1;
}

function CreateClassicOracle(n: Int, h: Int) : (Int) -> Bool {
    return ClassicOracle(n, h, _);
}


ClassicOracle, CreateClassicOracle

To test the classic oracle, we can use the following function that create the oracle for any given `h`, and prints its output for each possible input:


In [2]:
function TestClassicOracle(h: Int) : Unit {
    let n = 3; // number of bits
    let N = 1 <<< n; // 2^n: total number of integers
    
    // Create an oracle for the given hidden variable:
    let oracle = CreateClassicOracle(n, h);
    
    // Call the oracle for every possible value of X to see the values.
    for x in 0 .. N-1{
        Message($"{x}: {oracle(x)}");
    }    
}

TestClassicOracle

In [3]:
%simulate TestClassicOracle h=6

Oracle C (h:6, x:0) - 0.
0: False
Oracle C (h:6, x:1) - 0.
1: False
Oracle C (h:6, x:2) - 1.
2: True
Oracle C (h:6, x:3) - 1.
3: True
Oracle C (h:6, x:4) - 1.
4: True
Oracle C (h:6, x:5) - 1.
5: True
Oracle C (h:6, x:6) - 2.
6: False
Oracle C (h:6, x:7) - 2.
7: False


()

The actual classical algorithm follows:

In [4]:
function ClassicalAlgorithm(n: Int, oracle: (Int) -> Bool) : Int {
    mutable result = [false, size=n];

    for i in 0 .. n-1 {
        if (oracle(1 <<< i)) {
            set result w/= i <- true;
        }
    }

    let r = BoolArrayAsInt(result);
    Message($"Classical Result: {r}");

    return r;
}

operation RunClassicalAlgorithm(n: Int, h: Int) : Int {
    let oracle = CreateClassicOracle(n, h);
    let r = ClassicalAlgorithm(n, oracle);
    
    return r;
}

ClassicalAlgorithm, RunClassicalAlgorithm

A simple example of running the algorithm, it prints each time the oracle is called:

In [5]:
%simulate RunClassicalAlgorithm n=3 h=6

Oracle C (h:6, x:1) - 0.
Oracle C (h:6, x:2) - 1.
Oracle C (h:6, x:4) - 1.
Classical Result: 6


6

As expected, if we increase the  number of bits, the calls to the oracle increases accordingly:

In [6]:
%simulate RunClassicalAlgorithm n=30 h=432100

Oracle C (h:432100, x:1) - 0.
Oracle C (h:432100, x:2) - 0.
Oracle C (h:432100, x:4) - 1.
Oracle C (h:432100, x:8) - 0.
Oracle C (h:432100, x:16) - 0.
Oracle C (h:432100, x:32) - 1.
Oracle C (h:432100, x:64) - 1.
Oracle C (h:432100, x:128) - 1.
Oracle C (h:432100, x:256) - 1.
Oracle C (h:432100, x:512) - 1.
Oracle C (h:432100, x:1024) - 1.
Oracle C (h:432100, x:2048) - 0.
Oracle C (h:432100, x:4096) - 1.
Oracle C (h:432100, x:8192) - 0.
Oracle C (h:432100, x:16384) - 0.
Oracle C (h:432100, x:32768) - 1.
Oracle C (h:432100, x:65536) - 0.
Oracle C (h:432100, x:131072) - 1.
Oracle C (h:432100, x:262144) - 1.
Oracle C (h:432100, x:524288) - 0.
Oracle C (h:432100, x:1048576) - 0.
Oracle C (h:432100, x:2097152) - 0.
Oracle C (h:432100, x:4194304) - 0.
Oracle C (h:432100, x:8388608) - 0.
Oracle C (h:432100, x:16777216) - 0.
Oracle C (h:432100, x:33554432) - 0.
Oracle C (h:432100, x:67108864) - 0.
Oracle C (h:432100, x:134217728) - 0.
Oracle C (h:432100, x:268435456) - 0.
Oracle C (h:432100, x

432100


As we just saw, the classical solution takes $n$ calls to the oracle to find the solution. The quantum version requires only **1** call to the oracle to find the hidden value!

The quantum algorithm goes something like this:

```
   1. Prepare the state of the register to be in full super-position
   2. Apply the oracle once, to the phase of the register
   3. Undo the state preparation
```


The keys are:
1. by applying the oracle on a register prepared in a state of full super-position you can leverage quantum-parallelism to apply the oracle to all possible inputs.
2. when the oracle is applied to the phase, it creates interference on the state of the register and undoing the state preparation leaves only the hidden bits on.

The mathematical proof is out of scope for this workshop, but it is described [here](https://qiskit.org/textbook/ch-algorithms/bernstein-vazirani.html#1.3-The-Quantum-Solution--)

The implementation of the quantum oracle in Q# is very similar to the classical oracle:

In [7]:
// A Quantum Oracle. The main difference is that the oracle receives a qubit register as input
// and the result of the oracle is returned in another qubit, not the result of the operation.
operation QuantumOracle(n: Int, h: Int, x: Qubit[], result: Qubit) : Unit 
is Adj {
    let bits = IntAsBoolArray(h, n);

    for i in 0 .. n-1 {
        // Apply a controlled X, i.e. a conditional quantum application
        // only on the bits that are on the hidden variable.
        if (bits[i]) {
            Controlled X([x[i]], result);
        }
    }

    Message("Oracle Q");
}

function CreateQuantumOracle(n: Int, value: Int) : (Qubit[], Qubit) => Unit {
    return QuantumOracle(n, value, _, _);
}


QuantumOracle, CreateQuantumOracle

As with the classical case, we can write a simple test function that reports the return value of the oracle for each possible input.

Notice the input needs to be encoded in a quantum register, the output is also encoded as the value of a single qubit:

In [8]:
// Helper function that encodes an integer in the state of a quantum register
operation  EncodeIntOnQuantumRegister(value: Int, register: Qubit[]) : Unit 
is Adj {
    let n = Length(register);
    let bits = IntAsBoolArray(value, n);
    
    // Check every bit, flip the quantum bit that is on on the classical representation:
    for i in 0 .. n-1 {
        if (bits[i]) {
            X(register[i]);
        }
    }
}

// A method to call the oracle with each possible input and report the oracle value
operation TestQuanumOracle(h: Int) : Unit {
    let n = 3; // number of bits
    let N = 1 <<< n; // 2^n: total number of integers
    
    // Create a quantum oracle for the given hidden variable:
    let oracle = CreateQuantumOracle(n, h);
    
    // Call the oracle for every possible value of X to see the values.
    for i in 0 .. N-1{
        use x = Qubit[3];
        use y = Qubit();
        
        EncodeIntOnQuantumRegister(i, x);        
        oracle(x, y);
        let r = M(y);
        Message($"{i}: {r}");
        
        Adjoint EncodeIntOnQuantumRegister(i, x);
    }    
}

EncodeIntOnQuantumRegister, TestQuanumOracle

In [9]:
%simulate TestQuanumOracle h=6

Oracle Q
0: Zero
Oracle Q
1: Zero
Oracle Q
2: One
Oracle Q
3: One
Oracle Q
4: One
Oracle Q
5: One
Oracle Q
6: Zero
Oracle Q
7: Zero


()

As expected, the output of the oracle matches the classical values.

Now let's implement the quantum algorithm. We need a couple of helper functions to keep the main algorithm clean:

In [10]:
open Microsoft.Quantum.Diagnostics;

// A helper operation that prepares the register and results in full superposition.
operation PrepareState(register: Qubit[], result: Qubit) : Unit
is Adj {
    ApplyToEachA(H, register);
    // The result needs to be in |->, so it affects the phase.
    X(result);
    H(result);
}

// A helper function that prints the state of the quantum register with the given message.
function PrintState(debug: Bool, msg: String, register: Qubit[]) : Unit {
    if debug {
        Message(msg);
        DumpRegister((), register);
    }
}


PrepareState, PrintState

In [11]:
open Microsoft.Quantum.Measurement;

// The implementation of the quantum algorithm
operation QuantumAlgorithm(n: Int, oracle: (Qubit[], Qubit) => Unit, debug: Bool) : Result[] {
    use register = Qubit[n];
    use result = Qubit();

    // 1. Prepare state in super position:
    PrepareState(register, result);
    PrintState(debug, "State prepared in full-superposition", register);
    
    // 2. Apply the oracle **once**, notice the oracle affects the phase of the register.
    oracle(register, result);
    PrintState(debug, "Apply the oracle", register);
    
    // 3. Undo the state preparation by calling 'Adjoint'
    Adjoint PrepareState(register, result);
    PrintState(debug, "Undo state preparation", register);
    
    // Read the value from the register:
    let r = MultiM(register);
    if debug { Message($"Quantum Result: {r}"); }
    
    return r;
}

QuantumAlgorithm

In [18]:
// A simple driver that runs the quantum algorithm and returns its output:
operation RunQuantumAlgorithm(n: Int, h: Int, debug:Bool) : Result[] {
    let oracle = CreateQuantumOracle(n, h);
    
    let expected = IntAsBoolArray(h, n);
    Message($"h: {expected}\n");
    
    let actual = QuantumAlgorithm(n, oracle, debug);
    
    if debug {
        Message($"Expected: {expected}");
        Message($"Actual: {actual}");
    }
    
    return actual;
}

RunQuantumAlgorithm

Run the quantum algorithm with `debug` turned on. We're printing the state of the register on every step. Notice how the state changes 

1. $|0\rangle$
2. full-superposition
3. the bits reported by the oracle have a negative phase
4. the state of the hidden input

In [19]:
%simulate RunQuantumAlgorithm n=3 h=6 debug=true

h: [False,True,True]

State prepared in full-superposition


|0⟩	0.3535533905932738 + 0𝑖
|1⟩	0.3535533905932738 + 0𝑖
|2⟩	0.3535533905932738 + 0𝑖
|3⟩	0.3535533905932738 + 0𝑖
|4⟩	0.3535533905932738 + 0𝑖
|5⟩	0.3535533905932738 + 0𝑖
|6⟩	0.3535533905932738 + 0𝑖
|7⟩	0.3535533905932738 + 0𝑖

Oracle Q
Apply the oracle


|0⟩	0.3535533905932738 + 0𝑖
|1⟩	0.3535533905932738 + 0𝑖
|2⟩	-0.3535533905932738 + 0𝑖
|3⟩	-0.3535533905932738 + 0𝑖
|4⟩	-0.3535533905932738 + 0𝑖
|5⟩	-0.3535533905932738 + 0𝑖
|6⟩	0.3535533905932738 + 0𝑖
|7⟩	0.3535533905932738 + 0𝑖

Undo state preparation


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	-2.4651903288156602E-32 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	-3.6977854932234896E-32 + 0𝑖
|5⟩	3.6977854932234896E-32 + 0𝑖
|6⟩	1 + 0𝑖
|7⟩	-3.6977854932234896E-32 + 0𝑖

Quantum Result: [Zero,One,One]
Expected: [False,True,True]
Actual: [Zero,One,One]


Zero, One, One

Finally, run the algorithm with debug turned off to clearly see how many times the oracle is called. It is called only once regardless of the size of `n`.

In [29]:
%simulate RunQuantumAlgorithm n=15 h=23789 debug=false

h: [True,False,True,True,False,True,True,True,False,False,True,True,True,False,True]

Oracle Q


One, Zero, One, One, Zero, One, One, One, Zero, Zero, One, One, One, Zero, One

### Running on the cloud

As any other Q# program, it is easy to run this exact program on the cloud.

**To get started type `%azure.connect` on the cell below to insert the `%azure.connect` snippet.**

If you are running on a local instance of Jupyter Notebooks, you'll also need to manually type the workspace id and the location. These values can be found in the Overview section of the Workspace in the Azure Portal. See [%azure.connect](https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/azure.connect#examples-for-azureconnect) for details and examples.

Authenticated using Azure.Identity.AzureCliCredential


Connected to Azure Quantum workspace demo-sim in location westus2.


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,69
ionq.simulator,Available,3


Test by running on the simulator first:

In [31]:
%azure.target ionq.simulator

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.simulator,Available,3


In [32]:
%azure.submit RunQuantumAlgorithm  n=6 h=43 debug=false

Submitting RunQuantumAlgorithm to target ionq.simulator...
Job successfully submitted for 500 shots.
   Job name: RunQuantumAlgorithm
   Job ID: 1872971b-0014-44a9-a7ad-b8b4cb108ace


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
RunQuantumAlgorithm,1872971b-0014-44a9-a7ad-b8b4cb108ace,Executing,ionq.simulator,3/2/2022 7:47:57 AM +00:00,3/2/2022 7:47:59 AM +00:00,


In [33]:
%azure.status

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
RunQuantumAlgorithm,1872971b-0014-44a9-a7ad-b8b4cb108ace,Succeeded,ionq.simulator,3/2/2022 7:47:57 AM +00:00,3/2/2022 7:48:00 AM +00:00,3/2/2022 7:48:00 AM +00:00


In [34]:
%azure.output

Result,Frequency,Histogram
"[1,1,0,1,0,1]",1,


Now, run against the quantum device:

In [39]:
%azure.target ionq.qpu

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,194


In [38]:
%azure.submit RunQuantumAlgorithm  n=6 h=43 debug=false

Submitting Q2 to target ionq.qpu...
Job successfully submitted for 500 shots.
   Job name: Q2
   Job ID: 41a3abc9-4f3d-4dc9-87c1-b8f4f29f6ea7


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
Q2,41a3abc9-4f3d-4dc9-87c1-b8f4f29f6ea7,Waiting,ionq.qpu,3/2/2022 4:48:28 AM +00:00,,


This might take a while, as the QPU is really busy:

In [26]:
 %azure.status 41a3abc9-4f3d-4dc9-87c1-b8f4f29f6ea7

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
Q2,41a3abc9-4f3d-4dc9-87c1-b8f4f29f6ea7,Succeeded,ionq.qpu,3/2/2022 4:48:28 AM +00:00,3/2/2022 5:41:08 AM +00:00,3/2/2022 5:41:12 AM +00:00


In [28]:
%azure.output 41a3abc9-4f3d-4dc9-87c1-b8f4f29f6ea7

Result,Frequency,Histogram
"[1,1,0,0,0,0]",0.002,
"[1,0,0,1,0,0]",0.002,
"[1,1,0,1,0,0]",0.036,
"[1,1,0,0,0,1]",0.01,
"[1,0,0,1,0,1]",0.006,
"[0,1,0,1,0,1]",0.014,
"[1,1,0,1,0,1]",0.908,
"[1,1,0,1,1,1]",0.018,
"[0,0,0,0,0,0]",0.002,
"[1,0,0,0,0,0]",0.002,


Notice the output of the qpu reports some incorrect results, this is driven by the noise on existing quantum hardware that does not include any error correction mechanism.

# Next steps

I recommended visiting the [Quantum Katas](https://aka.ms/quantum-katas) to continue learning about Q# and quantum computing.

The quantum katas are a self-paced hands on exercises that help you learn about the basic elements of quantum computing and even advanced algorithms.

Visit https://aka.ms/quantum-katas to learn more.